In [3]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer 

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from sklearn.model_selection import cross_val_score

path_name = 'your_path_name_here/mental_health.csv'
df=pd.read_csv(path_name)
print(df)

df = df.drop_duplicates()

#Split the matrix into predictors and response 
msg=df["text"]
msg=msg.str.replace('[^a-zA-Z0-9]+'," h", regex = True) 
y=df['label'].values
y 


#Stemming and Tokenising
stemmer=PorterStemmer()
msg=msg.apply(lambda line:[stemmer.stem(token.lower()) for token in word_tokenize(line)]).apply(lambda token:" ".join(token))
msg=msg.apply(lambda line:[token for token in word_tokenize(line) if len(token)>2]).apply(lambda y:" ".join(y))

#Vectorising
tf=TfidfVectorizer() 
data_vec=tf.fit_transform(msg)

#Train, Validation, and Test Split ###RANDOMSTATE=1###
from sklearn.model_selection import train_test_split
#test_size=0.3 implies 70% will be used to training and 30% for testing. 
#random_state sets seed for random number generator, ensuring reducibility
x_train,x_test,y_train,y_test=train_test_split(data_vec,y,test_size=0.3,random_state=1)

                                                    text  label
0      dear american teens question dutch person hear...      0
1      nothing look forward lifei dont many reasons k...      1
2      music recommendations im looking expand playli...      0
3      im done trying feel betterthe reason im still ...      1
4      worried  year old girl subject domestic physic...      1
...                                                  ...    ...
27972  posting everyday people stop caring  religion ...      0
27973  okay definetly need hear guys opinion ive pret...      0
27974  cant get dog think ill kill myselfthe last thi...      1
27975  whats point princess bridei really think like ...      1
27976  got nudes person might might know snapchat do ...      0

[27977 rows x 2 columns]


In [4]:
#Logistic Lasso with 5 fold Cross Validation for Hyperparameter Tuning

alphas = 10.0 ** np.arange(-5, 5, 0.1)

cv_scores = [] 

# Perform cross-validation for each value of C
for alpha in alphas:
    modelLR_lasso = LogisticRegression(penalty='l1', C = 1/alpha, random_state=1, solver='liblinear')
    scores = cross_val_score(modelLR_lasso, x_train, y_train, cv = 5)  # 5-fold cross-validation
    cv_scores.append(np.mean(scores))


# Find the index corresponding to the best C value
best_CV_index = np.argmax(cv_scores)
best_alpha = alphas[best_CV_index]

# Inspect the best C value
print("Best C value selected by cross-validation:", 1/best_alpha)
print("Best lambda value (regularisation value) selected by cross-validation:", best_alpha)

modelLR_lasso = LogisticRegression(penalty='l1', C=1/best_alpha, random_state=1, solver = 'liblinear')

modelLR_lasso.fit(x_train,y_train)


# Make predictions on a new set of data
predictions = modelLR_lasso.predict(x_test)

# Compare the predicted labels to the true labels
accuracy_on_test_set = modelLR_lasso.score(x_test, y_test) #accuaracy = # of correct predictions / total # of predictions

print("Accuracy on the test set:", accuracy_on_test_set)

print(classification_report(y_test,predictions))

Best C value selected by cross-validation: 2.5118864315096747
Best lambda value (regularisation value) selected by cross-validation: 0.39810717055348227
[0 1 1 ... 0 1 0]
Accuracy on the test set: 0.9170638703527169
              precision    recall  f1-score   support

           0       0.90      0.94      0.92      4271
           1       0.93      0.90      0.91      4121

    accuracy                           0.92      8392
   macro avg       0.92      0.92      0.92      8392
weighted avg       0.92      0.92      0.92      8392

